In [26]:
family_name='897_.2407 (7)'
BASE = '03c009b451e012002513513a5d7e1b00_3444.trace.hooklog'

family_path='data/family/'+ family_name                  
directory_path = family_path +'/hooklogs/'
featureProfile = dict()

alidata_path = family_path +'/'+family_name+'_alignment.p' 
offset = 0
window = 1

In [ ]:
#profile => featureProfile.keys()
#profile_dict => featureProfile
#alignmentList_dict => alignment
#matchMatrix_dict => matchMatrix
#gapSeqList_dict => gapSeqList
#stageMatrix_dict => stageMatrix

In [27]:
# featureProfile => featureProfile{}

import os
%run "testDef_addFeature.ipynb"

for f in os.listdir(directory_path):

    if f.endswith('.trace.hooklog'):
        fh = FeatureHooklog(directory_path+ f)
        featureProfile[f] = fh.getHkli_noContainTS()
# featureProfile

In [28]:
# alignment => alignment_dict{}

import multiprocessing
import pickle

%run "classes/alignment/Alignment.ipynb"

print 'total ',len(featureProfile.keys()), 'Execution Traces'

if not os.path.isfile(alidata_path): 
    
    pool = multiprocessing.Pool(processes=7) #可以調整processes數量parallel computing
    
    ali = dict()
    for hk in featureProfile.keys():
        ali[hk] = pool.apply_async(pairwise_NW, (BASE, hk, 2, -1, -3, 1, ))
    pool.close()
    pool.join()
    
    alignment = dict()
    for t in ali:
        alignment[t] = ali[t].get()[2]
    pickle.dump(alignment, open(alidata_path, "wb"))

else:
    alignment = pickle.load(open(alidata_path,'rb')) #將alignment結果自路徑中的pickle檔讀取，並存至alignment
    print '(read alignment pickle)'
    
print 'all pairwise alignment done.'
# alignment

total  7 Execution Traces
(read alignment pickle)
all pairwise alignment done.


In [29]:
# matchMatrix => matchMatrix_dict 
# gapSeqList => gapSeqList_dict

# use => alignment, featureProfile

matchMatrix={i+1:[] for i in xrange(len(featureProfile[BASE]))} #index start from 1 to |PB|
gapSeqList={i:[] for i in xrange(len(featureProfile[BASE])+1)} #start is gap, end is not gap
 
for i,k in enumerate(alignment):
    print 'profile', k
    isMeet1 = 0
    if len(featureProfile[k]) > len(featureProfile[BASE]):
        for ii,((i1,t1),(i2,t2)) in enumerate(alignment[k]):
            #判斷是否有在PB[0]之前就插入
            if not isMeet1 and i1 == -1:
                strBuf_list = [t2]
                while alignment[k][ii+1][0][0] == -1:
                    strBuf_list.append(alignment[k][ii+1][1][1])
                    ii += 1
                gapSeqList[0].append((k, strBuf_list))
                isMeet1 = 1
            else:
                isMeet1 = 1
            
            #判斷PB[0]之後的情形
            if t1=='=' and t2!='=' and isMeet1 and alignment[k][ii-1][0][0]!=-1:
                lastIndex = alignment[k][ii-1][0][0]
                strBuf_list = [t2]
                try:
                    while alignment[k][ii+1][0][0] == -1:
                        strBuf_list.append(alignment[k][ii+1][1][1])
                        ii += 1
                except:
                    pass                
                gapSeqList[lastIndex].append((k, strBuf_list))
            elif t2=='=' and t1!='=':
                matchMatrix[i1].append((k,'gap','=')) 
            elif t1!='=' and t2!='=' and t1!=t2:
                matchMatrix[i1].append((k,'mismatch',t2))
            elif t2!='=' and t1!='=' and t1==t2:
                matchMatrix[i1].append((k,'match',t2))
                
    elif len(featureProfile[k]) <= len(featureProfile[BASE]):
        for ii,((i1,t1),(i2,t2)) in enumerate(alignment[k]):
            #判斷是否有在PB[0]之前就插入
            if not isMeet1 and i2 == -1:
                strBuf_list = [t1]
                while alignment[k][ii+1][1][0] == -1:
                    strBuf_list.append(alignment[k][ii+1][0][1])
                    ii += 1
                gapSeqList[0].append((k, strBuf_list))
                isMeet1 = 1
            else:
                isMeet1 = 1
            
            #判斷PB[0]之後的情形
            if t2=='=' and t1!='=' and isMeet1 and alignment[k][ii-1][1][0]!=-1:
                lastIndex = alignment[k][ii-1][1][0]
                strBuf_list = [t1]
                try:
                    while alignment[k][ii+1][1][0] == -1:
                        strBuf_list.append(alignment[k][ii+1][0][1])
                        ii += 1
                except:
                    pass
                gapSeqList[lastIndex].append((k, strBuf_list))
            elif t1=='=' and t2!='=':
                matchMatrix[i2].append((k,'gap','=')) 
            elif t2!='=' and t1!='=' and t1!=t2:
                matchMatrix[i2].append((k,'mismatch',t1))
            elif t2!='=' and t1!='=' and t1==t2:
                matchMatrix[i2].append((k,'match',t1))


profile 32abc951d6c707d2036c9e2a50e5fcb6_2868.trace.hooklog
profile 2272d05b959f133bcf3e38c48ddbfcad_3256.trace.hooklog
profile a0ceb22ed87df9019f39c5c135a58c17_3228.trace.hooklog
profile b295318cdd0ad09baa1cdadf86b90fc1_3228.trace.hooklog
profile 6123f4e765d6ba2bb4d9878de8f0598b_3200.trace.hooklog
profile 03c009b451e012002513513a5d7e1b00_3444.trace.hooklog
profile c32e9387f94c67a39d4c35b111302b0b_3208.trace.hooklog


In [30]:
# recheck data in structures are the same
# use => alignment, gapSeqList, matchMatrix
for k in alignment:
    print k, len(featureProfile[k])
    totalLen = 0
    for i in gapSeqList:
        for kin, l in gapSeqList[i]:
            if k == kin:
                totalLen += len(l)
                break
    for i in matchMatrix:
        for kin, stat, api in matchMatrix[i]:
            if k == kin and stat != 'gap':
                totalLen += 1
                break
    print 'totalLen:', totalLen

32abc951d6c707d2036c9e2a50e5fcb6_2868.trace.hooklog 7
totalLen: 7
2272d05b959f133bcf3e38c48ddbfcad_3256.trace.hooklog 7
totalLen: 7
a0ceb22ed87df9019f39c5c135a58c17_3228.trace.hooklog 7
totalLen: 7
b295318cdd0ad09baa1cdadf86b90fc1_3228.trace.hooklog 9
totalLen: 9
6123f4e765d6ba2bb4d9878de8f0598b_3200.trace.hooklog 7
totalLen: 7
03c009b451e012002513513a5d7e1b00_3444.trace.hooklog 6
totalLen: 6
c32e9387f94c67a39d4c35b111302b0b_3208.trace.hooklog 7
totalLen: 7


In [31]:
# stageMatrix => stageMatrix_dict (merge first)
# use => featureProfile, gapSeqList, matchMatrix

stageMatrix = {k:[] for k in featureProfile}
for i,k in enumerate(featureProfile):
    strBuf_list = []
    for ii in range(len(featureProfile[BASE])+1):
        if ii == 0:
            #有在gapSeqList的話就加入，k對應的list，否則加入gap
            if len(gapSeqList[0]) > 0:
                isGap = 1
                for hk, l in gapSeqList[0]:
                    if k == hk:
                        stageMatrix[k].append(l)
                        isGap = 0
                        break
                if isGap:
                    stageMatrix[k].append(['=']) 
        else:
            a = [api for hk, stat, api in matchMatrix[ii] if k==hk][0]
            strBuf_list.append(a)
            
            if len(gapSeqList[ii]) > 0:
                stageMatrix[k].append(strBuf_list)
                strBuf_list = []       
                
                isGap = 1
                for hk, l in gapSeqList[ii]:
                    if k == hk:
                        stageMatrix[k].append(l)
                        isGap = 0
                        break
                if isGap:
                    stageMatrix[k].append(['='])                     
            else:
                if ii <= len(matchMatrix)-1:
                    curColStat_list = [stat for kin, stat, api in matchMatrix[ii]]
                    nextColStat_list = [stat for kin, stat, api in matchMatrix[ii+1]]
                    if curColStat_list != nextColStat_list:
                        stageMatrix[k].append(strBuf_list)
                        strBuf_list = []
                    if curColStat_list == nextColStat_list and ii == len(matchMatrix)-1:
                        stageMatrix[k].append(strBuf_list)

# stageMatrix

In [23]:
# stageMatrix => stageMatrix_dict (transform multiple gap segment into one gap segment)

for i,hk in enumerate(stageMatrix):
    for ii,seg in enumerate(stageMatrix[hk]):
        if set(seg) == set(['=']):
            stageMatrix[hk][ii] = ['=']
# 有必要？ bug?

In [38]:
%run "testDef_stageMatrix.ipynb"

s = StageMatrix('897_.2407 (7)')
# s.getFeatureProfile()
# s.getAlignment()
print s
s1 = set(stageMatrix)
s2 = set(s.getStageMatrix())
print s1.symmetric_difference(s2)

---1 featrureProfile fin ---
	total  7 Execution Traces (read alignment pickle)
	all pairwise alignment done.
---2 alignment fin ---
---3 match& gap fin ---
---4 stageMatrix fin ---
class stageMatrix, 897_.2407 (7), BASE = 03c009b451e012002513513a5d7e1b00_3444.trace.hooklog,
 path = data/family/897_.2407 (7), offset = 0, windows = 1
set([])
